## Underlying map creation code

In [272]:
# import libraries
import numpy as np
import pandas as pd
! pip install folium
import folium
from folium.features import DivIcon
import matplotlib.cm as cm
import matplotlib.colors as colors
import random
import json
import re

Imports successful!


### Map Formatting Code - Colors, Notes, Title, Legend

In [273]:
# static colors
def colors():
    return [
        "#FF0000", "#008941", "#0000FF", "#6A0DAD", "#7F7F7F", "#FF6600", "#FFCC00", "#006FA6", "#A30059",
        "#E75480", "#7A4900", "#0000A6", "#63FFAC", "#B79762", "#004D43", "#8FB0FF", "#997D87",
        "#5A0007", "#809693", "#FEFFE6", "#1B4400", "#4FC601", "#3B5DFF", "#4A3B53", "#FF2F80",
        "#61615A", "#BA0900", "#6B7900", "#00C2A0", "#FFAA92", "#FF90C9", "#B903AA", "#D16100",
        "#DDEFFF", "#000035", "#7B4F4B", "#A1C299", "#300018", "#0AA6D8", "#013349", "#00846F",
        "#372101", "#FFB500", "#C2FFED", "#A079BF", "#CC0744", "#C0B9B2", "#C2FF99", "#001E09",
        "#00489C", "#6F0062", "#0CBD66", "#EEC3FF", "#456D75", "#B77B68", "#7A87A1", "#788D66",
        "#885578", "#FAD09F", "#FF8A9A", "#D157A0", "#BEC459", "#456648", "#0086ED", "#886F4C",
        "#34362D", "#B4A8BD", "#00A6AA", "#452C2C", "#636375", "#A3C8C9", "#FF913F", "#938A81",
        "#575329", "#00FECF", "#B05B6F", "#8CD0FF", "#3B9700", "#04F757", "#C8A1A1", "#1E6E00",
        "#7900D7", "#A77500", "#6367A9", "#A05837", "#6B002C", "#772600", "#D790FF", "#9B9700",
        "#549E79", "#FFF69F", "#201625", "#72418F", "#BC23FF", "#99ADC0", "#3A2465", "#922329",
        "#5B4534", "#FDE8DC", "#404E55", "#0089A3", "#CB7E98", "#A4E804", "#324E72", "#6A3A4C",
        "#83AB58", "#001C1E", "#D1F7CE", "#004B28", "#C8D0F6", "#A3A489", "#806C66", "#222800",
        "#BF5650", "#E83000", "#66796D", "#DA007C", "#FF1A59", "#8ADBB4", "#1E0200", "#5B4E51",
        "#C895C5", "#320033", "#FF6832", "#66E1D3", "#CFCDAC", "#D0AC94", "#7ED379", "#012C58"
    ]

In [274]:
# function to put a title on the map
def add_title(folium_map, title):
    title_html = '''
                 <h3 align="center" style="font-size:20px;font-family:Serif"><b>{}</b></h3>
                 '''.format(title)   
    folium_map.get_root().html.add_child(folium.Element(title_html))
    return folium_map

In [275]:
# function to put notes and sources on the map
def add_notes_sources(folium_map, notes, sources):   
    
    notes_sources_html = f"""
    <div id=notes_sources>
       <b>Notes:</b><br>{notes}<br>
       <b>Sources:</b><br>{sources}
    </div>
    """
    
    script = f"""
        <script type="text/javascript">
        var oneTimeExecution = (function() {{
                    var executed = false;
                    return function() {{
                        if (!executed) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-bottom leaflet-left').length) || (!executed)) {{
                                          document.getElementsByClassName('leaflet-bottom leaflet-left')[0].innerHTML += `{notes_sources_html}`;
                                          clearInterval(checkExist);
                                          executed = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        oneTimeExecution()
        </script>
      """
    
    css = """
    <style type='text/css'>
      #notes_sources {
          margin:0 -9999rem;
          padding: 1rem 10000rem;
          font-size:12px;
          font-family:Serif;
          background:white;
          position:relative;
          color:black;
      }
      
      @media print {
        #notes_sources {
          background:white !important;
        }
      }
      
    </style>
    """
    
    folium_map.get_root().header.add_child(folium.Element(css+ script))
    return folium_map

In [276]:
# function to put a legend for the longitude/latitude data on the map
def add_sa_legend(folium_map, legend_info, len_lv_set):
    
    i = 1
    legend_categories = ""
    square_css = ""
    media_square_css = ""
    for lv, va in zip(legend_info['Legend_Value'], legend_info['Value']):
        if len_lv_set == 1:
            legend_categories += f"<li><span id='square1'></span>{lv}</li>"
            square_css = f"\n#square1" + "{\nborder:1px solid grey; background:orange; width:7px; height:7px; display: inline-block;}"
            media_squre_css = f"\n#square1" + "{background-color:orange !important;-webkit-print-color-adjust: exact;}"
        elif len_lv_set == 2:
            legend_categories += f"<li><span id='square{i}'></span>{lv}</li>"
            if va == 1:
                square_css += f"\n#square{i}" + "{\nborder:1px solid grey; background:#FFFFB7; width:7px; height:7px; display: inline-block;}"
                media_square_css += f"\n#square{i}" + "{background-color:#FFFFB7 !important;-webkit-print-color-adjust: exact;}"
            else:
                square_css += f"\n#square{i}" + "{\nborder:1px solid grey; background:red; width:7px; height:7px; display: inline-block;}"
                media_square_css += f"\n#square{i}" + "{background-color:red !important;-webkit-print-color-adjust: exact;}"
            i += 1  
        else:
            legend_categories += f"<li><span id='square{i}'></span>{lv}</li>"
            if va == 1:
                square_css += f"\n#square{i}" + "{\nborder:1px solid grey; background:#FFFFB7; width:7px; height:7px; display: inline-block;}"
                media_square_css += f"\n#square{i}" + "{background-color:#FFFFB7 !important;-webkit-print-color-adjust: exact;}"
            elif va == 2:
                square_css += f"\n#square{i}" + "{\nborder:1px solid grey; background:orange; width:7px; height:7px; display: inline-block;}"
                media_square_css += f"\n#square{i}" + "{background-color:orange !important;-webkit-print-color-adjust: exact;}"
            else:
                square_css += f"\n#square{i}" + "{\nborder:1px solid grey; background:red; width:7px; height:7px; display: inline-block;}"
                media_square_css += f"\n#square{i}" + "{background-color:red !important;-webkit-print-color-adjust: exact;}"
            i += 1 
            
    legend_html = f"""
    <div id='maplegend' class='maplegend'>
      <div class='legend-title'>Service Areas</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legend_categories}
        </ul>
      </div>
    </div>
    """
    
    script = f"""
        <script type="text/javascript">
        var oneTimeExecution = (function() {{
                    var executed = false;
                    return function() {{
                        if (!executed) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!executed)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          executed = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        oneTimeExecution()
        </script>
      """
    
    css = """

    <style type='text/css'>
      .maplegend {
        z-index:9999;
        float:left;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:15px;
        font-style:Serif;
        positon: relative;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 70%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 60%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 60%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    """ + square_css + """
     
     @media print {
         .maplegend {
            background-color: rgba(255, 255, 255, 1) !important;
         }
    """ + media_square_css + """
     }
     
     </style>
    """

    folium_map.get_root().header.add_child(folium.Element(script + css))
    return folium_map

In [277]:
# function to put a legend for the longitude/latitude data on the map
def add_latlong_legend(folium_map, title, colorz, labels, radii_list):
    
    # lat/long legend
    if len(colorz) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_by_label = dict(zip(labels, colorz))
    
    i = 1
    legend_categories = ""
    dot_css = ""
    media_dot_css = ""
    for label, color in color_by_label.items():
        legend_categories += f"<li><span id='dot{i}'></span>{label}</li>"
        dot_css += f"\n#dot{i}" + "{\nborder:1px solid black; background:" + f"{color};" + "width:7px; height:7px; border-radius:50%; display: inline-block;}"
        media_dot_css += f"\n#dot{i}" + "{background-color: " + f"{color}" + " !important;-webkit-print-color-adjust: exact;}"
        i += 1
        
    # buffer legend if provided
    if str(radii_list) != "['nan']":
        for i, r in enumerate(radii_list):
            r = r.replace(" ","")
            color = colors()[i]
            legend_categories += f"<li><span style='border:1px solid {color};background-color:#FFFFFF;height:10px;border-radius:50%;width:10px;'></span>{r} Mile Radius</li>"
            
    legend_html = f"""
    <div id='maplegend' class='maplegend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legend_categories}
        </ul>
      </div>
    </div>
    """
    
    script = f"""
        <script type="text/javascript">
        var oneTimeExecution = (function() {{
                    var executed = false;
                    return function() {{
                        if (!executed) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-left').length) || (!executed)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-left')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-left')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-left')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          executed = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        oneTimeExecution()
        </script>
      """
   
    css = """

    <style type='text/css'>
      .maplegend {
        z-index:9999;
        float:left;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:15px;
        font-style:Serif;
        positon: relative;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 70%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 60%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 60%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    """ + dot_css + """
     
     @media print {
         .maplegend {
            background-color: rgba(255, 255, 255, 1) !important;
         }
    """ + media_dot_css + """
     }
     
     </style>
    """

    folium_map.get_root().header.add_child(folium.Element(script + css))
    return folium_map

### Map Creation Code

In [288]:
# function to loop over starting lat/longs to make multiple maps - although it can also be used to just make one map
def make_maps(map_starts=None):
    
    # check that the maps_start file is provided
    if map_starts == None:
        print("ERROR! You must supply an Excel file with you Maps Start Data.")
        return

    # read in map_start data
    ms_df = pd.read_excel("{}".format(map_starts), sheet_name="Start_Maps")
    
    # check to make sure all required map_starts columns are in the data file and all values are valid
    needed_columns = ['Latitude','Longitude','Title','Zoom', 'Notes', 'Sources', 'Radii', 'Map_Style', 'Lat_Long_File', 'Choro_File', 'SA_File']
    map_starts_check1 = check_map_starts_columns_present(ms_df,needed_columns)
    if map_starts_check1 != "Good":
        print(map_starts_check1)
        return
    map_starts_check2 = check_map_starts_values_valid(ms_df)
    if map_starts_check2 != "Good":
        print(map_starts_check2)
        return

    # make all "map starts" maps in a loop. 
    for lat, lng, title, notes, sources, zoom, radii, map_style, lat_long_file, choro_file, sa_file in zip(ms_df['Latitude'], ms_df['Longitude'], ms_df['Title'], ms_df['Notes'], ms_df['Sources'], ms_df['Zoom'], ms_df['Radii'], ms_df['Map_Style'], ms_df['Lat_Long_File'], ms_df['Choro_File'], ms_df['SA_File']):

        # initialize a blank map
        m = make_blank_map(lat, lng, zoom, map_style) 
            
        # if choropleth data provided
        if str(choro_file) != "nan":  
            
            # read in choropleth data
            try:
                choro_df = pd.read_excel(choro_file, sheet_name="Data")
            except:
                print("ERROR! Error reading in Choro_File. Check that your filepath is correct and that your data sheet is named Data.")
                return
                
            # check to make sure all required Choro_Files columns are present
            needed_columns = ['Geography','Geography_Type','Color','Value','Bins','Legend_Name']
            choro_check1 = check_choro_columns_present(choro_df,needed_columns)
            if choro_check1 != "Good":
                print(choro_check1)
                return
            # validate Choro_Files data field values
            choro_check2 = check_choro_values_valid(choro_df)
            if choro_check2 != "Good":
                print(choro_check2)
                return
            
            # add choropleth fills to base map
            m = add_geo_fills(m, choro_df)
        
        # if service area data provided
        if str(sa_file) != "nan":
            
            # read in service area data
            try:
                sa_df = pd.read_excel(sa_file, sheet_name="Data")
            except:
                print("ERROR! Error reading in your SA_File. Check that your filepath is correct and your data sheet is named Data.")
                return            
            
            # check to make sure all required Lat_Long_File columns are present
            needed_columns = ['Geography','Geography_Type','Legend_Value']
            sa_check1 = check_sa_columns_present(sa_df,needed_columns)
            if sa_check1 != "Good":
                print(sa_check1)
                return
            # validate SA_File data field values
            sa_check2 = check_sa_values_valid(sa_df)
            if not isinstance(sa_check2, set):
                print(sa_check2)
                return 
            
            # add service area fills to base map
            m = add_sa_fills(m, sa_df, sa_check2)
            
        # if lat/long data provided
        if str(lat_long_file) != "nan":
            
            # read in lat/long data
            try:
                ll_df = pd.read_excel(lat_long_file, sheet_name="Data")
            except:
                print("ERROR! Error reading in your Lat_Long_File for plotting coordinates. Check that your filepath is correct and your data sheet is named Data.")
                return
                
            # check to make sure all required Lat_Long_File columns are present
            needed_columns = ['Legend_Info','Label','Latitude','Longitude','Point_Size','Color_Map_File']
            latlong_check1 = check_latlong_columns_present(ll_df,needed_columns)
            if latlong_check1 != "Good":
                print(latlong_check1)
                return
            # validate Lat_Long_File data field values
            latlong_check2 = check_latlong_values_valid(ll_df)
            if latlong_check2 != "Good":
                print(latlong_check2)
                return  
 
            # add coordinates to map
            m = add_coordinates_to_map(m, ll_df, lat, lng, radii)
            
        # if radii data provided, plot radii buffers
        try:
            radii_list = str(radii).split(',')
            for i, r in enumerate(radii_list):
                r_meters = int(r.strip())*1609.34
                color = colors()[i]
                folium.Circle(location=[lat, lng],
                              radius=r_meters,
                              color=color,
                              weight=1.5).add_to(m)
                
        except:
            print("No radii buffers provided. If you entered Radii values, make sure they are in the correct format.") 
            
        # clean notes and sources if they are empty
        if str(notes) == "nan":
            notes = ""
        if str(sources) == "nan":
            sources = ""
            
        # add title, notes, and sources
        m = add_notes_sources_title(m, str(notes), str(sources), str(title))
        
        # save the map
        m.save("../output/html/{}.html".format(title.replace("<br>","-").replace("/","-").replace("\\","-"))) 


In [289]:
# function to initialize a blank folium map
def make_blank_map(start_lat, start_long, zs, tiles_type):
    m = folium.Map(location=[start_lat, start_long], zoom_start=zs, tiles=tiles_type, width=1000, height=900)
    return m

In [1]:
# function to fill in choropleth geographic areas
def add_geo_fills(m, df):
    
    # clean geo data before using it in choropleth map
    df, geo_dict, key_on = clean_geo_data(df)
    
    if isinstance(df['Bins'][0],str):
        j = df['Bins'][0]
        b = [float(x) for x in j.split(',')]
    else:
        b = df['Bins'][0]
    
    # add choropleth data to map
    fc = df['Color'][0]
    folium.Choropleth(
        geo_data=geo_dict,
        data=df,
        columns=['Geography', 'Value'],
        key_on='feature.'+key_on,
        fill_color=fc,
        bins=b,
        fill_opacity=0.6, 
        line_opacity=0.2,
        legend_name=df['Legend_Name'][0],
        reset=True 
        ).add_to(m)
    
    # return map
    return m 

In [291]:
# function to fill in service areas
def add_sa_fills(m, df, lv_set):
    
    # clean geo data before using it in choropleth map
    df, geo_dict, key_on = clean_geo_data(df)
    
    # create dictionary for colors
    df['Value'] = 1
    len_lv_set = len(lv_set)
    lv_list = list(lv_set)
    counter = 1
    if len_lv_set == 3:
        for i in lv_list:
            if "Overlap" in i or "Combined" in i:
                df.loc[df['Legend_Value'] == i, 'Value'] = 2
            else:
                df.loc[df['Legend_Value'] == i, 'Value'] = counter
                counter += 2
    else:
        for i in lv_list:
            df.loc[df['Legend_Value'] == i, 'Value'] = counter
            counter += 1
    
    # for legend
    legend_info = df[['Legend_Value', 'Value']].drop_duplicates()

    # add service area data to map
    choropleth = folium.Choropleth(
                    geo_data=geo_dict,
                    data=df,
                    columns=['Geography', 'Value'],
                    key_on='feature.'+key_on,
                    fill_color='YlOrRd',
                    bins=3,
                    fill_opacity=0.6, 
                    line_opacity=0.2,
                    reset=True 
                 )
   
    # remove choropleth legend
    for key in choropleth._children:
        if key.startswith('color_map'):
            del(choropleth._children[key])
    choropleth.add_to(m)

    # add legend
    m = add_sa_legend(m, legend_info, len_lv_set)
    
    # return map
    return m

In [292]:
# function to plot longitude/latitude data on the map
def add_coordinates_to_map(m, df, lat, lng, radii):
    
    # define color map if provided
    color_map = df["Color_Map_File"][0]
    
    # sort by legend info
    df.sort_values(by=['Legend_Info'], inplace=True)
    
    # find number of colors to use for points
    unique_legend = df['Legend_Info'].unique()
    color_no = np.count_nonzero(unique_legend)

    # encode categorical Legend_Info
    df["Legend_Info"] = df["Legend_Info"].astype('category')
    df["Legend_Info_Cat"] = df["Legend_Info"].cat.codes
    
    # set color scheme for the different points on map based on Legend_Info
    rainbow_user_provided = False
    if str(color_map) == "nan":
        rainbow = colors()[:color_no]
    else:
        # read in Color_Map_File if provided
        try: 
            colors_map_df = pd.read_excel(color_map, sheet_name="Color_Map")
        except:
            print("ERROR! Error reading in Color_Map_File. Check that your filepath is correct and that your data sheet is named Color_Map.")
            return m
        # check to make sure all required Color_Map_File columns are present      
        color_map_check1 = check_colormap_columns_present(colors_map_df, ['Legend_Info','Color'])       
        if color_map_check1 != "Good":
            print(color_map_check1)
            return m
        # validate Color_Map_File data field values 
        color_map_check2 = check_colormap_values_valid(colors_map_df)       
        if color_map_check2 != "Good":
            print(color_map_check2)
            return m
        # merge on Color_Map_File to Lat_Long_File by Legend_Info
        df_len_1 = df.shape[0]
        df = df.merge(colors_map_df, on='Legend_Info')
        df_len_2 = df.shape[0]
        # alert if not all Legend_Info unique values are accounted for
        if df_len_1 != df_len_2:
            color_map_check3 = set(unique_legend)-set(colors_map_df['Legend_Info'].unique())
            print("ERROR! Your Lat_Long_File has the values {} but your Color_Map_File does not give a color for these Legend_Info values. The points associated with these Legend_Info values will not show up on your map. Make sure to map all unique Legend_Info values from your Lat_Long_File to a hex colors in your Color_Map_File.".format(color_map_check3))
            return m
        rainbow_user_provided = True

    # plot for non-user provided colors
    if rainbow_user_provided == False:
        for lati, lon, point_size, group in zip(df['Latitude'], df['Longitude'], df['Point_Size'], df['Legend_Info_Cat']):
            folium.CircleMarker(
                [lati, lon],
                radius=point_size,
                weight=1,
                fill=True,
                color="black",
                fill_opacity=100,
                fill_color=rainbow[group]).add_to(m)
        # add legend
        legend_labels = df.sort_values(by=['Legend_Info_Cat'])['Legend_Info'].unique()
    
    # plot for user provided colors
    else:
        for lati, lon, point_size, group in zip(df['Latitude'], df['Longitude'], df['Point_Size'], df['Color']):
            folium.CircleMarker(
                [lati, lon],
                radius=point_size,
                weight=1,
                fill=True,
                color="black",
                fill_opacity=100,
                fill_color=group).add_to(m)
        # add legend
        labels_and_colors = df.sort_values(by=['Legend_Info'])[['Legend_Info', 'Color']].drop_duplicates()
        legend_labels = labels_and_colors['Legend_Info']
        rainbow = labels_and_colors['Color']
        
    # add labels
    for lati, lon, label in zip(df['Latitude'], df['Longitude'], df['Label']):
        if label != "Blank":
            folium.map.Marker(
                [lati+0.0002, lon-0.0002],
                icon = DivIcon(
                    icon_size=(150,36),
                    icon_anchor=(0,0),
                    html='<div style="font-size:6pt;font-style:Serif;color:black;font-weight:bold;">%s</div>' % label,
                )
            ).add_to(m)
        
    # add coordinate legend
    try:
        radii_list = str(radii).split(',') 
        m = add_latlong_legend(m, 'Legend', rainbow, legend_labels, radii_list)
    except:
        m = add_latlong_legend(m, 'Legend', rainbow, legend_labels, "")
        
    # return map
    return m

### Data Validation Functions

In [278]:
# check that all Map_Start data fields are present
def check_map_starts_columns_present(df, needed_columns):
    for column in df.columns:
        if column in needed_columns:
            needed_columns.remove(column)
    if len(needed_columns) > 0:
        return "ERROR! You must supply an Excel file for your map start data with all of the following columns: Title, Notes, Sources, Latitude, Longitude, Zoom, Radii, Map_Style, Lat_Long_File, Choro_File, and SA_File." 
    return "Good"  

In [279]:
# check that Map_Starts data values are valid
def check_map_starts_values_valid(df):
    for lat, long, title, notes, sources, z, radii, ms, lat_long_file, choro_file, sa_file in zip(df['Latitude'], df['Longitude'], df['Title'], df['Notes'], df['Sources'], df['Zoom'], df['Radii'], df['Map_Style'], df['Lat_Long_File'], df['Choro_File'], df['SA_File']):
        # check that values that must be provided are provided
        if (str(lat) == "nan" or str(long) == "nan" or str(z) == "nan" or str(ms) == "nan"):
            return "ERROR! Latitude, Longitude, Zoom, and Map_Style in your Map Starts Excel file must be provided in each row." 
        # check that only choro_file or sa_file is provided, but not both
        if str(sa_file) != "nan" and str(choro_file) != "nan":
            return "ERROR! You can only provide values for either Choro_File or SA_File but not both. Please see the documentation."
        # check that values are the correct types
        if (not isinstance(lat,float) or not isinstance(long,float)):
            return "ERROR! Latitude and Longitude in your Map Starts Excel file must be floats in each row."
        if (not isinstance(z,int) or (isinstance(z,int) and (z < 1 or z > 20))):
            return "ERROR! Zoom in your Map Starts Excel file must be an integer between 1 and 20."
        if ms not in ['Open Street Map','Stamen Terrain','Stamen Toner', 'stamenwatercolor', 'Cartodb Positron']:
            return "ERROR! The Map_Style in your Map Starts Excel file must be either Open Street Map, Stamen Terrain, Stamen Toner, stamenwatercolor or Cartodb Positron."
    return "Good"

In [280]:
# check that all Choro_File data fields are present
def check_choro_columns_present(df, needed_columns):
    for column in df.columns:
        if column in needed_columns:
            needed_columns.remove(column)
    if len(needed_columns) > 0:
        return "ERROR! You must supply the Choro_File with all of the following columns: Geography, Geography_Type, Color, Value, Bins, and Legend_Name."
    return "Good"

In [281]:
# check that all Choro_File values are valid
def check_choro_values_valid(df):
    gt_same = df['Geography_Type'][0]
    c_same = df['Color'][0]
    b_same = df['Bins'][0]
    ln_same = df['Legend_Name'][0]
    # check Geography_Type and Color are in pre-defined types
    if gt_same not in ['Countries', 'States', 'MSAs', 'Counties', 'Zips']:
        return "ERROR! Geography_Type in your Choro_File must be one of the following: Countries, States, MSAs, Counties, Zips. Make sure the value is spelled exactly as shown here."
    if c_same not in ['Blues','Greens','BuGn','BuPu','GnBu','OrRd','PuBu','PuBuGn','PuRd','RdPu','YlGn','YlGnBu','YlOrBr','YlOrRd']:
        return "ERROR! Color in your Choro_File must be one of the following: Blues, Greens, BuGn, BuPu, GnBu, OrRd, PuBu, PuBuGn, PuRd, RdPu, YlGn, YlGnBu, YlOrBr, YlOrRd"
    # check Bins is correct
    if isinstance(b_same,str):
        try:
            b_test = [int(x) for x in b_same.split(',')]
        except:
            return "ERROR! If you supply a list of values to separate your Bins in your Choro_File, they must all be integers."
    else:
        b_test = b_same.item()
    if not isinstance(b_test,int) and not isinstance(b_test,list):
        return "ERROR! You must either have an integer Bin value that will separate your data in to x different bins or a list of integer values that will separate your values in your Choro_File."
    if isinstance(b_test, int) and b_test < 3:
        return "ERROR! The number of Bins in your Choro_File must be >= 3 if you supply an integer value."
    if isinstance(b_test, list) and len(b_test) < 3:
        return "ERROR! You must have at least 3 separate values in your Choro_File if you supply Bins as a list of values."
    # check other
    for g, gt, color, bins, v, ln in zip(df['Geography'],df['Geography_Type'],df['Color'],df['Bins'],df['Value'],df['Legend_Name']):
        # check that values that must be provided are provided
        if (str(g) == "nan" or str(gt) == "nan" or str(color) == "nan" or str(bins) == "nan" or str(v) == "nan" or str(ln) == "nan"):
            return "ERROR! Geography, Geography_Type, Color, Bins, Value, and Legend_Name must be provided in each row in your Choro_File." 
        # check that columns that must have the same value for each row do indeed have the same value
        if (gt_same != gt or c_same != color or b_same != bins or ln_same != ln):
            return "ERROR! Geography_Type, Color, Bins, and Legend_Name must have the same value for all rows in your Choro_File."         
    return "Good"  

In [282]:
# check that all SA_File data fields are present
def check_sa_columns_present(df, needed_columns):
    for column in df.columns:
        if column in needed_columns:
            needed_columns.remove(column)
    if len(needed_columns) > 0:
        return "ERROR! You must supply the SA_File with all of the following columns: Geography, Geography_Type, and Legend_Value."
    return "Good"

In [283]:
# check that SA_File data values are valid
def check_sa_values_valid(df):
    gt_same = df['Geography_Type'][0]
    lv_set = set()
    # check Geography_Type is in a pre-defined type
    if gt_same not in ['Countries', 'States', 'MSAs', 'Counties', 'Zips']:
        return "ERROR! Geography_Type in your SA_File must be one of the following: Countries, States, MSAs, Counties, Zips. Make sure the value is spelled exactly as shown here."
    # check other
    for g, gt, lv in zip(df['Geography'], df['Geography_Type'], df['Legend_Value']):
        # check that values that must be provided are provided
        if (str(g) == "nan" or str(gt) == "nan" or str(lv) == "nan"):
            return "ERROR! Geography, Geography_Type, and Legend_Vaue in your SA_File must be provided in each row." 
        # check that columns that must have the same value for each row do indeed have the same value
        if gt_same != gt:
            return "ERROR! Geography_Type must have the same value for all rows in your SA_File."
        # check that a max of three Legend_Values are provided
        if lv not in lv_set:
            lv_set.add(lv)
            if len(lv_set) > 3:
                return "ERROR! Legend_Value can have a maximum of 3 unique values. Please see the documentation with example."
    return lv_set

In [284]:
# check that all Lat_Long_File data fields are present
def check_latlong_columns_present(df, needed_columns):
    for column in df.columns:
        if column in needed_columns:
            needed_columns.remove(column)
    if len(needed_columns) > 0:
        return "ERROR! You must supply the Lat_Long_File with all of the following columns: Legend_Info, Label, Longitude, Latitude, Point_Size, and Color_Map_File."
    return "Good"

In [285]:
# check that Lat_Long_File data values are valid
def check_latlong_values_valid(df):
    cmf_same = df['Color_Map_File'][0]
    for label, li, lat, long, ps, cmf in zip(df['Label'],df['Legend_Info'],df['Latitude'],df['Longitude'],df['Point_Size'],df['Color_Map_File']):
        # check that values that must be provided are provided
        if (str(label) == "nan" or str(li) == "nan" or str(lat) == "nan" or str(long) == "nan" or str(ps) == "nan"):
            return "ERROR! Label, Legend_Info, Latitude, Longitude, and Point_Size in your Lat_Long_File must be provided in each row." 
        # check that values are the correct types
        if (not isinstance(lat,float) or not isinstance(long,float)):
            return "ERROR! Latitude and Longitude in your Lat_Long_File must be floats in each row."
        if not isinstance(ps,int): 
            return "ERROR! Point_Size in your Lat_Long_File must be an integer in each row."
        if isinstance(ps,int) and (ps < 1 or ps > 20): 
            return "ERROR! Point_Size in your Lat_Long_File must be an integer between 1 and 20."
        # check that columns that must have the same value for each row do indeed have the same value
        if (str(cmf_same) != str(cmf)):
            return "ERROR! Color_Map_File must have the same value for all rows in your Lat_Long_File. This value can be empty if you want to use default colors for your points."        
    return "Good"

In [286]:
# check that all Color_Map_File data fields are present
def check_colormap_columns_present(df, needed_columns):
    for column in df.columns:
        if column in needed_columns:
            needed_columns.remove(column)
    if len(needed_columns) > 0:
        return "ERROR! You must supply the Color_Map_File with all of the following columns: Legend_Info and Color."
    return "Good"

In [287]:
# check that Color_Map_File data values are valid
def check_colormap_values_valid(df):
    for li, c in zip(df['Legend_Info'], df['Color']):
        # check that values that must be provided are provided
        if (str(li) == "nan" or str(c) == "nan"):
            return "ERROR! Legend_Info and Color in your Color_Map_File must be provided in each row." 
        # check that colors are hex values
        if not re.search(r'^#(?:[0-9a-fA-F]{3}){1,2}$', c):
            return "ERROR! Color in your Color_Map_File must be a hex color value in each row." 
    return "Good"

### Map Creation Helper Functions

In [293]:
# function to clean geography data before using it in a choropleth or service area map
def clean_geo_data(df):
    
    # grab the geography type and the corresponding geo_json file
    geo_type = df['Geography_Type'][0]
    geo_json = r'../geo_json/' + geo_type + '.json'
    f = open(geo_json, 'r')
    f.close()

    # make sure key_on df column is a string
    df['Geography'] = df['Geography'].astype(str)
    if geo_type == "Zips":
        # ensure there are no more than 1,500 zip codes
        if df.shape[0] > 1500:
            print("ERROR! You can only have a maximum of 1,500 zip codes.")
            return m
        # add leading zeros to zips
        df.Geography = df.Geography.str.zfill(5)
    
    # clean dataframe columns
    input_geographies = df['Geography'].tolist()
    
    # remove geo json geographies that are not present in the data frame
    with open(geo_json,'r') as f:
        
        data = json.load(f)
        
        # countries / MSAs
        if geo_type in ['Countries','MSAs']:
            key_on = "properties.name"
            geo_dict = limit_geo_json(geo_type,input_geographies,data,key_on)
            
        # states
        if geo_type in ['States','CBSAs']:
            key_on = "properties.NAME"
            geo_dict = limit_geo_json(geo_type,input_geographies,data,key_on)
          
        # counties
        if geo_type == 'Counties':
            key_on = "id"
            geo_dict = limit_geo_json(geo_type,input_geographies,data,key_on)
           
        # zips
        if geo_type == 'Zips':
            key_on = "properties.ZCTA5CE10"
            geo_dict = limit_geo_json(geo_type,input_geographies,data,key_on)
            
    return df, geo_dict, key_on

In [294]:
# function to limit geo_json dictionary to just what user has in his/her file
def limit_geo_json(geo_type, input_geographies,data,key_on):
    geo_dict = {"type":"FeatureCollection","features":[]}
    geographies = data['features']
    for g in geographies:
        if geo_type in ['States', 'Countries','MSAs','Zips']:
            key_on_first = key_on.split(".")[0]
            key_on_second = key_on.split(".")[1]
            if g[key_on_first][key_on_second] in input_geographies:
                geo_dict['features'].append(g)
        elif geo_type in ['Counties']:
            if g[key_on] in input_geographies:
                geo_dict['features'].append(g)
    return geo_dict

In [295]:
# function to add notes, sources, and title to the final map
def add_notes_sources_title(m, notes, sources, title):
    m = add_title(m, title)
    m = add_notes_sources(m, notes, sources)
    return m